class CGraphSage():
    
    def __init__(pos, root, fct = lambda x: x[0]*x[1] , transform=None, pre_transform=None, size=1002, skip=100):
    """
    root = where the dataset should be stored. 
    This folder is split into raw_dir (downloaded dataset) and processed_dir (processed data)

    """
    #super(RGGDataset, self).__init__(root, transform, pre_transform)
    self.fct = fct #this is the function that we use for sampling things
    self.size = size
    self.skip = skip
    self.root = root

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import pickle #for saving and loading the dataset

import numpy as np
import networkx as nx
import torch
from torch_geometric.data import Data
import time


# In[2]:


import random
from torch_geometric.utils import from_networkx

import matplotlib.pyplot as plt

# In[13]:


# For plotting

# To avoid copying things to GPU memory,
# ideally allocate everything in torch on the GPU
# and avoid non-torch function calls
torch.set_printoptions(precision=10) # Set displayed output precision to 10 digits

from torchquad import enable_cuda # Necessary to enable GPU support
from torchquad import Trapezoid, Simpson, Boole, MonteCarlo, VEGAS # The available integrators
import torchquad


# In[3]:


from torch_geometric.nn import SAGEConv

enable_cuda() # Use this to enable GPU support

09:32:22|TQ-INFO| Initializing torchquad.
09:32:23|TQ-INFO| __pyTorch VERSION:<module 'torch.version' from '/home/math/maskey/anaconda3/envs/pyg_cuda102/lib/python3.8/site-packages/torch/version.py'>
09:32:23|TQ-INFO| __CUDNN VERSION:7605
09:32:23|TQ-INFO| __Number of CUDA Devices:1
09:32:23|TQ-INFO| Active CUDA Device: GPU0
09:32:23|TQ-INFO| Setting default tensor type to cuda.Float32 (CUDA is initialized).


import sys
sys.path.insert(1,'../src')
from DataLoader import RGGDataset

DL = RGGDataset(root = '../input')

data = DL.get(101)

pos = data.pos

In [18]:
N = 1002 #How large shall the graphs become?
skip = 100 #Should we consider all graphs, or only every skip'th
r = 0.2

eucl_norm = lambda x,y: torch.sqrt(torch.pow(x,2) + torch.pow(y, 2))

def indicator_fct(x, y, distance, center):
    z = (eucl_norm(x-center[0], y- center[1]).item() < distance)*1
    return z

k = lambda x,y: x*y

simp = Simpson()  # Initialize Simpson solver

In [19]:
def forward(positions, weight, fct=k):
    """
    pipeline should be as follows in the main:
    1. load model and its weights
    2. load graph data and its positions
    3. apply forward to return a torch.tensor with the c-GraphSage output for every position
    """
    k = fct
    g = lambda x,y, distance, center: indicator_fct(x, y, distance, center)*fct(x,y)
    values = []
    the_weight = weight
    
    for i in range(0, len(positions)):
        h = lambda x: g(x[:,0], x[:,1], r, positions[i])
        l = lambda x: indicator_fct(x[:,0],x[:,1], r, positions[i])
        
        int_of_f = simp.integrate(h,dim=2,N=1000000,integration_domain = [[0,1],[0,1]])
        measure_Ball = simp.integrate(l,dim=2,N=1000000,integration_domain = [[0,1],[0,1]])
        
        values.append(torch.tensor(the_weight*(int_of_f/measure_Ball)))
    return torch.tensor(values)
        
        
    """
    for key, value in position.items():
        h = lambda x: g(x[:,0], x[:,1], r, torch.tensor(value))
        l = lambda x: indicator_fct(x[:,0],x[:,1], r, torch.tensor(value))
        int_of_f = simp.integrate(h,dim=2,N=1000000,integration_domain = [[0,1],[0,1]])
        measure_Ball = simp.integrate(l,dim=2,N=1000000,integration_domain = [[0,1],[0,1]])
        error2 = the_weight*(int_of_f/measure_Ball) - gnn_output[key]
        error = error + torch.pow(error2, 2)
    error = torch.sqrt( (1/length)*error )
    return error

    """

In [20]:
output = forward( pos[0], weight = 1, fct = k)

09:36:03|TQ-INFO| Integration grid created.


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [6]:
import sys 

sys.path.insert(1,'../src')

from DataLoader import RGGDataset

DL = RGGDataset(root = '../input')

data = DL.get(101)

pos = data.pos

In [7]:
pos

tensor([[0.1549290866, 0.7097089887],
        [0.1823909134, 0.8313137293],
        [0.7967348099, 0.1866734475],
        [0.5944018364, 0.4262868166],
        [0.9569894075, 0.3034688830],
        [0.3752178848, 0.6063228846],
        [0.5812447071, 0.8066115379],
        [0.2091544122, 0.4697555900],
        [0.5836423635, 0.1162658259],
        [0.6426201463, 0.3657320738],
        [0.6794329882, 0.6360018253],
        [0.8379440308, 0.0899395123],
        [0.0311105270, 0.1365505755],
        [0.1719858944, 0.2885262370],
        [0.2078580707, 0.4456683993],
        [0.7622871995, 0.2488225549],
        [0.5342204571, 0.4125072062],
        [0.8627122045, 0.2245300114],
        [0.6719439626, 0.7563374639],
        [0.3537233770, 0.1843595356],
        [0.3427940309, 0.4616136849],
        [0.8043653965, 0.4093938768],
        [0.4219855964, 0.0581574552],
        [0.7615956664, 0.0024481448],
        [0.3800218105, 0.6268571615],
        [0.1602447629, 0.3205794990],
        [0.4